In [ ]:
### 'Cohere Embed English' is the market’s leading multimodal (text, image) representation model used for semantic search, retrieval-augmented generation (RAG), classification, and clustering. Embed English has top performance on the HuggingFace MTEB benchmark and performs well on a variety of industries such as Finance, Legal, and General-Purpose Corpora.The model was trained on nearly 1B English training pairs.

In [118]:
# Load environment variables for API keys from .env file
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

## Text Embeddings
### Generating Text Embeddings and Performing Vector Similarity Search in Redis

In [119]:
# This code creates an embeddings client using Azure AI Inference SDK, connects to the Cohere-embed-v3-english model hosted on Github Marketplace using GitHub token,
# and generates embeddings for three sample phrases. It then prints the embeddings vector length and first/last values
# for each phrase, along with usage statistics.
import os

from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Cohere-embed-v3-english"
token = os.environ["GITHUB_TOKEN"]

client = EmbeddingsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token)
)

response = client.embed(
    input=["first phrase", "second phrase", "third phrase"],
    model=model_name
)

for item in response.data:
    length = len(item.embedding)
    print(
        f"data[{item.index}]: length={length}, "
        f"[{item.embedding[0]}, {item.embedding[1]}, "
        f"..., {item.embedding[length-2]}, {item.embedding[length-1]}]"
    )
print(response.usage)

data[0]: length=1024, [0.0013399124, -0.01576233, ..., 0.007843018, 0.000238657]
data[1]: length=1024, [0.036590576, -0.0059547424, ..., 0.011405945, 0.004863739]
data[2]: length=1024, [0.04196167, 0.029083252, ..., -0.0027484894, 0.0073127747]
{'prompt_tokens': 6, 'completion_tokens': 0, 'total_tokens': 6}


In [120]:
# A function to generate embedding for a given string
import os

from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Cohere-embed-v3-english"
token = os.environ["GITHUB_TOKEN"]

def generate_embeddings_for_text(text):
    client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
    )

    response = client.embed(
        input=[text],
        model=model_name
    )

    return response.data[0].embedding

# Example usage
generate_embeddings_for_text("This is a very happy person")

[0.05505371,
 0.012062073,
 -0.035064697,
 -0.10284424,
 0.0058021545,
 -0.015419006,
 -0.04824829,
 0.0049438477,
 -0.0031261444,
 0.07562256,
 -0.020767212,
 -0.012008667,
 0.049804688,
 0.0033626556,
 -0.02268982,
 -0.0124435425,
 0.030715942,
 0.01586914,
 0.02734375,
 -0.004623413,
 0.011924744,
 0.03201294,
 -0.030288696,
 -0.034423828,
 0.013366699,
 -0.019592285,
 -0.00047922134,
 0.012939453,
 -0.028244019,
 -0.03982544,
 -4.4882298e-05,
 0.053009033,
 0.015045166,
 0.0061798096,
 -0.011405945,
 -0.015975952,
 -0.03451538,
 0.0005617142,
 0.004573822,
 0.009384155,
 0.028259277,
 -0.03074646,
 -0.12121582,
 0.0047836304,
 -0.115234375,
 0.002822876,
 -0.007259369,
 0.047821045,
 0.013641357,
 -0.02784729,
 -0.019821167,
 0.0027160645,
 -0.030273438,
 -0.01927185,
 -0.0024528503,
 0.06021118,
 0.001080513,
 -0.038269043,
 0.011749268,
 0.0018920898,
 0.020996094,
 0.050445557,
 0.023391724,
 0.028640747,
 -0.034118652,
 -0.0058670044,
 -0.054626465,
 -0.00013506413,
 0.03909301

In [121]:
# A function to create index in Redis
import redis
from redis.commands.search.field import TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

r = redis.Redis(host="localhost", port=6379)

INDEX_NAME = "textindex"                              # Vector Index Name
DOC_PREFIX = "textdoc:"                               # RediSearch Key Prefix for the Index

def create_index(vector_dimensions: int):
    try:
        # check to see if index exists
        r.ft(INDEX_NAME).info()
        print("Index already exists!")
    except:
        # schema
        schema = (
            TagField("tag"),                       # Tag Field Name
            VectorField("vector",                  # Vector Field Name
                "FLAT", {                          # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",             # FLOAT32 or FLOAT64
                    "DIM": vector_dimensions,      # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.HASH)

        # create Index
        r.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

In [122]:
# Create index in Redis

# define vector dimensions
VECTOR_DIMENSIONS = 1024

# create the index
create_index(vector_dimensions=VECTOR_DIMENSIONS)

In [123]:
# Add a documents to the index

import numpy as np

# instantiate a redis pipeline
pipe = r.pipeline()

# define some dummy data
objects = [
    {"name": "1", "tag": "foo", "message": "This is a very happy person"},
    {"name": "2", "tag": "moo", "message": "This is a happy dog"},
    {"name": "3", "tag": "bar", "message": "This is a sunny day"},
    {"name": "4", "tag": "car", "message": "Happiness is a state of mind"},
    {"name": "5", "tag": "baz", "message": "Happy employees work better"},
]

# write data
for obj in objects:
    # define key
    key = f"{DOC_PREFIX}{obj['name']}"
    # create a vector using cohere model
    obj["vector"] = np.array(generate_embeddings_for_text(obj["message"]), dtype=np.float32).tobytes()
    # HSET
    pipe.hset(key, mapping=obj)

res = pipe.execute()


In [124]:
# KNN queries

query = (
    Query("*=>[KNN 2 @vector $vec as score]")
     .sort_by("score")
     .return_fields("id", "message", "tag", "name", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vec": np.array(generate_embeddings_for_text("That is a happy person"), dtype=np.float32).tobytes()
}
r.ft(INDEX_NAME).search(query, query_params).docs

[Document {'id': 'textdoc:1', 'payload': None, 'score': '0.0745350122452', 'message': 'This is a very happy person', 'tag': 'foo', 'name': '1'},
 Document {'id': 'textdoc:2', 'payload': None, 'score': '0.232981920242', 'message': 'This is a happy dog', 'tag': 'moo', 'name': '2'}]

In [125]:
# Range queries

query = (
    Query("@vector:[VECTOR_RANGE $radius $vec]=>{$YIELD_DISTANCE_AS: score}")
     .sort_by("score")
     .return_fields("id", "message", "tag", "name", "score")
     .paging(0, 3)
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vec": np.array(generate_embeddings_for_text("That is a happy person"), dtype=np.float32).tobytes()
}
r.ft(INDEX_NAME).search(query, query_params).docs

[Document {'id': 'textdoc:1', 'payload': None, 'score': '0.0745350122452', 'message': 'This is a very happy person', 'tag': 'foo', 'name': '1'},
 Document {'id': 'textdoc:2', 'payload': None, 'score': '0.232981920242', 'message': 'This is a happy dog', 'tag': 'moo', 'name': '2'},
 Document {'id': 'textdoc:4', 'payload': None, 'score': '0.356992065907', 'message': 'Happiness is a state of mind', 'tag': 'car', 'name': '4'}]

# Image embeddings

In [ ]:
# This code converts an image file into a base64-encoded string using the ImageEmbeddingInput class from the Azure AI Inference SDK.
# The encoded string is then extracted from the ImageEmbeddingInput object and saved to a JSON file named 'image_data.json'.
# This base64 string can be used in REST API requests to generate embeddings for the image.
import json
from azure.ai.inference.models import ImageEmbeddingInput

# Load and encode the image
img_obj = ImageEmbeddingInput.load(image_file="sample1.png", image_format="png")
image_data = img_obj.__dict__['_data']['image']

with open('image_data.json', 'w') as f:
    json.dump(image_data, f)

In [145]:
# Generate image embedding using python code
import os

from azure.ai.inference import ImageEmbeddingsClient
from azure.ai.inference.models import ImageEmbeddingInput
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Cohere-embed-v3-multilingual"
token = os.environ["GITHUB_TOKEN"]

def generate_embedding_for_image(image_file):
    client = ImageEmbeddingsClient(
        endpoint=endpoint, 
        credential=AzureKeyCredential(token)
        )

    response = client.embed(
        model=model_name,
        input=[ImageEmbeddingInput.load(image_file=image_file, image_format="jpeg")]
        )

    # Print the length and first/last values of the embedding just to examine the output
    for item in response.data:
        length = len(item.embedding)
        print(
            f"data[{item.index}]: length={length}, [{item.embedding[0]}, {item.embedding[1]}, "
            f"..., {item.embedding[length-2]}, {item.embedding[length-1]}]"
        )

    return response.data[0].embedding

#generate_embedding_for_image("sample1.png")
generate_embedding_for_image("gallery/bradpitt1.jpeg")

data[0]: length=1024, [-0.03060913, 0.008811951, ..., 0.020431519, 1.1384487e-05]


[-0.03060913,
 0.008811951,
 0.0044174194,
 0.0022678375,
 -0.011314392,
 0.004825592,
 -0.006164551,
 -0.042816162,
 -0.038970947,
 0.038635254,
 0.031585693,
 -0.027359009,
 0.029418945,
 -0.02935791,
 0.057525635,
 -0.0016613007,
 -0.026382446,
 -0.0050849915,
 -0.09094238,
 -0.07183838,
 0.01448822,
 0.05316162,
 0.059692383,
 0.10296631,
 0.020889282,
 0.04083252,
 0.049926758,
 -0.03439331,
 0.080200195,
 -0.011146545,
 0.00027036667,
 -0.04562378,
 0.0065574646,
 0.061676025,
 0.024536133,
 0.062469482,
 -0.0074539185,
 -0.034118652,
 -0.031707764,
 0.0647583,
 0.039520264,
 0.0013008118,
 -0.028320312,
 0.037963867,
 -0.007282257,
 0.0044174194,
 0.020019531,
 0.025634766,
 -0.06311035,
 0.04397583,
 0.017929077,
 0.015701294,
 0.0134887695,
 0.01398468,
 0.031280518,
 0.03765869,
 0.052978516,
 -0.025604248,
 0.021560669,
 0.013916016,
 -0.015312195,
 -0.08154297,
 -0.03427124,
 -0.030181885,
 0.018356323,
 0.0039100647,
 -3.9219856e-05,
 -0.004901886,
 -0.027145386,
 0.009475

In [146]:
import redis
from redis.commands.search.field import TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

r = redis.Redis(host="localhost", port=6379)

IMG_INDEX_NAME = "imageindex"                              # Vector Index Name
IMG_DOC_PREFIX = "imagedoc:"                               # RediSearch Key Prefix for the Index

def create_index_for_images(vector_dimensions: int):
    try:
        # check to see if index exists
        r.ft(IMG_INDEX_NAME).info()
        print("Index already exists!")
    except:
        # schema
        schema = (
            VectorField("vector",                  # Vector Field Name
                "FLAT", {                          # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",             # FLOAT32 or FLOAT64
                    "DIM": vector_dimensions,      # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[IMG_DOC_PREFIX], index_type=IndexType.HASH)

        # create Index
        r.ft(IMG_INDEX_NAME).create_index(fields=schema, definition=definition)

In [147]:
# Create index for images in Redis

# define vector dimensions
VECTOR_DIMENSIONS = 1024

# create the index
create_index_for_images(vector_dimensions=VECTOR_DIMENSIONS)

In [148]:
import os

from azure.ai.inference import ImageEmbeddingsClient
from azure.ai.inference.models import ImageEmbeddingInput
from azure.core.credentials import AzureKeyCredential
import glob

endpoint = "https://models.inference.ai.azure.com"
model_name = "Cohere-embed-v3-multilingual"
token = os.environ["GITHUB_TOKEN"]

img_objects = [
    {"ks": "1", "name": "Brad Pitt", "path": "gallery/bradpitt1.jpeg"},
    {"ks": "2", "name": "Brad Pitt",  "path": "gallery/bradpitt2.jpeg"},
    {"ks": "3", "name": "Cameron Diaz", "path": "gallery/camerondiaz1.jpg"},
    {"ks": "4", "name": "Cameron Diaz", "path": "gallery/camerondiaz2.jpg"},
    {"ks": "5", "name": "Cameron Diaz", "path": "gallery/camerondiaz3.jpg"},
    {"ks": "6", "name": "Jennifer Lawrence", "path": "gallery/jennifer1.jpeg"},
    {"ks": "7", "name": "Jennifer Lawrence", "path": "gallery/jennifer2.jpeg"},
    {"ks": "8", "name": "Tom Cruise", "path": "gallery/tomcruise1.jpeg"},
    {"ks": "9", "name": "Tom Cruise", "path": "gallery/tomcruise2.jpeg"}
]

# write data
for img_obj in img_objects:
    # define key
    key = f"{IMG_DOC_PREFIX}{img_obj['ks']}"
    # create a vector using cohere model
    img_obj["vector"] = np.array(generate_embedding_for_image(img_obj["path"]), dtype=np.float32).tobytes()
    # HSET
    r.hset(key, mapping=img_obj)


data[0]: length=1024, [-0.03060913, 0.008811951, ..., 0.020431519, 1.1384487e-05]
data[0]: length=1024, [0.0013694763, 0.034057617, ..., 0.01739502, -0.036376953]
data[0]: length=1024, [-0.01348114, 0.024291992, ..., -0.034057617, 0.017471313]
data[0]: length=1024, [-0.019134521, -0.009407043, ..., -0.007347107, 0.0034446716]
data[0]: length=1024, [0.0039711, 0.03265381, ..., -0.018829346, 0.010009766]
data[0]: length=1024, [0.00484848, 0.028778076, ..., -0.010025024, 0.017608643]
data[0]: length=1024, [0.00907135, 0.025772095, ..., -0.01348114, 0.013015747]
data[0]: length=1024, [-0.022537231, 0.008720398, ..., 0.011001587, -0.029006958]
data[0]: length=1024, [-0.026687622, -0.0021362305, ..., 0.015357971, -0.035186768]


In [153]:
# KNN queries

query = (
    Query("*=>[KNN 3 @vector $vec as score]")
     .sort_by("score")
     .return_fields("id", "name", "path", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vec": np.array(generate_embedding_for_image("search_celebrity2.jpeg"), dtype=np.float32).tobytes()
}
r.ft(IMG_INDEX_NAME).search(query, query_params).docs


data[0]: length=1024, [-0.017410278, 0.020202637, ..., 0.020858765, -0.03817749]


[Document {'id': 'imagedoc:8', 'payload': None, 'score': '0.0731337666512', 'name': 'Tom Cruise', 'path': 'gallery/tomcruise1.jpeg'},
 Document {'id': 'imagedoc:9', 'payload': None, 'score': '0.0777180790901', 'name': 'Tom Cruise', 'path': 'gallery/tomcruise2.jpeg'}]

In [ ]:
# Range queries

query = (
    Query("@vector:[VECTOR_RANGE $radius $vec]=>{$YIELD_DISTANCE_AS: score}")
     .sort_by("score")
     .return_fields("id", "name", "path", "score")
     .paging(0, 3)
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.22,
    "vec": np.array(generate_embedding_for_image("search_celebrity2.jpeg"), dtype=np.float32).tobytes()
}
r.ft(IMG_INDEX_NAME).search(query, query_params).docs

data[0]: length=1024, [-0.017410278, 0.020202637, ..., 0.020858765, -0.03817749]


[Document {'id': 'imagedoc:8', 'payload': None, 'score': '0.0731337666512', 'name': 'Tom Cruise', 'path': 'gallery/tomcruise1.jpeg'},
 Document {'id': 'imagedoc:9', 'payload': None, 'score': '0.0777180790901', 'name': 'Tom Cruise', 'path': 'gallery/tomcruise2.jpeg'},
 Document {'id': 'imagedoc:2', 'payload': None, 'score': '0.237340867519', 'name': 'Brad Pitt', 'path': 'gallery/bradpitt2.jpeg'}]